In [2]:
import pdb
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.insert(0, '../')
from reservoirpy.datasets import japanese_vowels
from reservoirpy import set_seed, verbosity
from reservoirpy.observables import nrmse, rsquare
from reservoirpy.nodes import Reservoir, RidgeRegression, Input
from sklearn.metrics import accuracy_score
from reservoirpy.mat_gen import fast_spectral_initialization, generate_input_weights

import ipywidgets as widgets
set_seed(42)
verbosity(0)

0

In [3]:
X_train, Y_train, X_test, Y_test = japanese_vowels()

In [13]:
hyper = {
    "N": 500,
    "sr": 0.7,
    "leak": 9e-2,
    "iss": 1e-3,
    "isd": 5e-3,
    "isd2": 5e-3,
    "ridge": 1e-3, #1e-8
    "n_mfcc": 13,
    "mfcc": True,
    "d": False,
    "dd": False,
    "alpha":1.0,
    "seed": 12345689
}


In [14]:
def get_weights(hyper):
    rng = np.random.default_rng(hyper["seed"])
    Wins = []
    input_bias = True
    if hyper["mfcc"]:
        win = generate_input_weights(hyper["N"], hyper["n_mfcc"], proba=0.2,
                                     input_bias=input_bias, seed=rng)
        input_bias = False
        Wins.append(win)
    if hyper["d"]:
        wd = generate_input_weights(hyper["N"], hyper["n_mfcc"], proba=0.2,
                                    input_bias=input_bias, seed=rng)
        input_bias = False
        Wins.append(wd)
    if hyper["dd"]:
        wdd = generate_input_weights(hyper["N"], hyper["n_mfcc"], proba=0.2,
                                     input_bias=input_bias, seed=rng)
        input_bias = False
        Wins.append(wdd)

    Win = np.hstack(Wins)

    W = fast_spectral_initialization(hyper["N"], sr=hyper["sr"], seed=rng, proba=0.2)
    return Win, W

In [15]:
source = Input()
W, Win = get_weights(hyper)
reservoir = Reservoir(N=hyper["N"], sr=hyper["sr"], lr=hyper["leak"], W=W, Win=Win)
readout = RidgeRegression(alpha=hyper['alpha'])
model = source >> reservoir >> readout

In [16]:
states_train = []
for x in X_train:
    states = reservoir.run(x, reset=True)
    states_train.append(states[-1, np.newaxis])

# the ridge regression class expects inputs and outputs as a numpy 2d matrix
# thus we do a bit of pre-processing where we convert the states_train list 
# and Y_train into a numpy array and remove any unecessary dimensions
readout.fit(np.array(states_train).squeeze(), np.array(Y_train).squeeze())

IndexError: tuple index out of range

In [ ]:
Y_test = np.array(Y_test).squeeze()
Y_pred_class = np.argmax(y_pred, axis=1)
Y_test_class = np.argmax(Y_test, axis=1)
score = accuracy_score(Y_test_class, Y_pred_class)
print("Accuracy: ", f"{score * 100:.3f} %")